# Sampleo de parámetros el modelo VAR

La finalidad del primer sampleo es encontrar los parámetros para el modelo Periódico Vectorial Autoregresivo. Si consideramos una serie de tiempo de dimensión $K$, $y_{1},..., y_{T}$ en donde $y_{t}=(y_{1t}, ..., y_{Kt})$ es un vector de dimensión $K$, entonces el modelo VAR$(p)$ queda definido por:

$$y_{t}=\upsilon + A_{1}y_{t-1}+...+A_{p}y_{t-p}+u_{t}$$

En donde $\upsilon=(\upsilon_{1},...,\upsilon_{K})$ es el vector de interceptos ($K\times1$), $A_{i}$ son los coeficientes matriciales ($K\times K$) y finalmente $u_{t}$ es un ruido blanco no-singular de covarianza $\Sigma_{u}$, la cual es de dimensión ($K\times K$). En este caso, asumimos el ruido blanco como una normal multivariada de $K$ dimensiones.

Específicamente, se utilizó Gibbs Sampling considerando las distribuciones de probabilidad dadas por:

$$p(A_{ijk}|A_{-i-j-k}, \Sigma_{u}, D), \forall i,j,k. 1\leq i \leq p \wedge 1\leq j, k \leq K$$

para la matriz $A=(A_{1}, ..., A_{p})$ y:

$$p({\Sigma_{u}}_{ij}|A, {\Sigma_{u}}_{-i-j}, D), \forall i,j. 1 \leq i, j \leq K$$

Es decir, se fijan todas las entradas de las matrices $A$ o $\Sigma_{u}$ excepto una, y esta es sampleada según alguna distribución $q$ predefinida. Hasta el momento hemos elegido $q$ ser una normal univariada.

Cabe destacar que para cada sampleo univariado se revisa si es que el nuevo parámetro (en conjunto con los anteriores) cumplen con ciertas restricciones. Para la matriz $A$ se debe cumplir que esta sea estable y $\Sigma_{u}$ debe ser semidefinida positiva (porque se trata de una normal multivariada), por lo tanto, no se acepta el sampleo de un nuevo parámetro hasta que se cumplen estas condiciones.

En la implementación definimos una variable $\theta$ que contiene todos los parámetros correspondientes a las matrices $A$ y $\Sigma_{u}$ en un vector. De esta manera pudimos ir iterando por cada parámetro para ir sampleándolos de manera más simple. 

In [ ]:
import utils as ut
import pickle
import scipy.stats as stats

DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
STATS_PATH = '/home/chrisams/Documents/datasets/data_TAIM/stats.pickle'
#DATA_PATH = 'C:/Users/Samuel/Desktop/2018_01/IIC 3695 Intel Maquina/Tareas y Proyecto/Proyecto/Code/TAIM/data'
q = stats.norm(loc=0, scale=1)
K = 3
p = 1
iters = 2
debug = False
n_rows = 10000 # Number of rows of the data to load

samples = ut.gibbs_sampling(iters, DATA_PATH, K, p, q, n_rows=n_rows, debug=debug)

In [ ]:
print(samples[0][0])
print(samples[0][1])

# Sampleo de velocidades desde el modelo VAR

Un segundo sampleo se hace con la función $\texttt{sim_wind}$. Esta función toma una de las muestras de los coeficientes $A$ y de la matriz $\Sigma_{u}$ obtenida en el proceso anterior y, para cada tiempo $t$ que se quiere predecir (definido en la variable $\texttt{horizon}$) genera velocidades eólicas sampleando desde el ruido blanco cuya matriz de covarianza corresponde a $\Sigma_{u}$. De esta forma, para cada tiempo $t$ se obtiene una distribución de las velocidades en ese instante.

Un ejemplo de esto se puede ver más abajo en los gráficos. Se eligieron los primeros parámetros sampleados por el método anterior y muestrearon 10 velocidades para cada tiempo $t$ que se quería predecir (por temas de visualización). 

In [ ]:
import numpy as np
x0 = np.transpose(np.matrix([[0.3,0.4,0.7]]))
A = samples[0][0]
CovU = samples[0][1]

print(CovU)
eigval = np.linalg.eigvals(CovU)
print(eigval)
#print(np.all(np.isreal(eigval)))

horizon = 50
n_samples = 10
time_series = ut.sim_wind(A,CovU,x0,horizon,n_samples)

# Adjust time series with their true mean and std.
with open(STATS_PATH, 'rb') as fp:
    stats_dict = pickle.load(fp)

for i in range(K):
    mean = stats_dict['plant_{}'.format(i)]['mean']
    std = stats_dict['plant_{}'.format(i)]['std']
    time_series[i] = time_series[i] * std + mean

#Grafica series de tiempos resultantes
ut.plot_series(time_series)
